## **Install the packages**

## **Import the libraries**

In [583]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
import seaborn as sns
import yfinance as yf


## **Download the data**

In [584]:
# Select the stocks to test the model
stocks = ['ORCL','EL','NOW','ADI','ISRG','CVS','T','EOG','REGN','TJX']
stocks_compl = ["AAPL"] + stocks

# Download the data for first stock and store it
apl = yf.Ticker("AAPL")
aapl = apl.history(period ='5y')

# Calculate the returns for the first stock and store it
pct_aapl = aapl["Close"].pct_change()

prices_aapl = aapl["Close"]
returns = pct_aapl.to_frame()
prices = prices_aapl.to_frame()

# Store the generated returns and downloaded data
returns = returns.rename(columns={"Close": stocks_compl[0]})
prices = prices.rename(columns={"Close": stocks_compl[0]})

# Downloading the stocks' data using the 'yfinance' library
for stock in stocks:
    df1 = yf.Ticker(stock)
    df = df1.history(period='5y')

    # Calculate the percent change
    df_pct = df["Close"].pct_change()
    df_price = df["Close"]

    # Store the returns and prices
    returns = returns.join(df_pct).rename(columns={"Close": stock})
    prices = prices.join(df_price).rename(columns={"Close": stock})

# Remove the empty cells from the returns dataframe
returns = returns.dropna()

# **RECURSIVE BISECTION**

In [585]:
def get_rec_bipart(cov, sort_ix):

    # compute HRP allocation

    # intialize weights of 1
    w = pd.Series(1, index=sort_ix)

    # intialize all items in one cluster
    c_items = [sort_ix]
    while len(c_items) > 0:
        c_items = [i[int(j):int(k)] for i in c_items for j,k in
        ((0,len(i)/2),(len(i)/2,len(i))) if len(i)>1]
        # now it has 2

        for i in range(0, len(c_items), 2):
            c_items0 = c_items[i] # cluster 1
            c_items1 = c_items[i+1] # cluter 2
            c_var0 = get_cluster_var(cov, c_items0)
            c_var1 = get_cluster_var(cov, c_items1)
            alpha = 1 - c_var0/(c_var0+c_var1)
            w[c_items0] *= alpha
            w[c_items1] *=1-alpha
    return w

## **METHODS TO COMPARE THE STRATEGY**

In [586]:
# Compute the weights according to 'mean variance' method
def compute_mean_varience_weights(covariances):
    inv_covar = np.linalg.inv(covariances)
    u = np.ones(len(covariances))
    x = np.dot(inv_covar, u) / np.dot(u, np.dot(inv_covar, u))
    return pd.Series(x, index = stocks_compl, name="Mean varience")

# Compute the weights according to 'risk parity' method
def compute_risk_parity_weights(covariances):
    weights = (1 / np.diag(covariances))
    x = weights / sum(weights)
    return pd.Series(x, index = stocks_compl, name="Risk parity")

# Compute the weights according to 'Monte Carlo' method
def compute_monte_carlo_weights(covariances):
    x = [1 / len(covariances) for i in range(len(covariances))]
    return pd.Series(x, index = stocks_compl, name="Monte Carlo")



## **Store the weights generated from different methods**

In [587]:
# Compute the weights according to 'hrp' method
cov = returns.cov()

# Make a seperate dataframe to store the expected returns generated by all the methods used

weights_mean_varience = compute_mean_varience_weights(cov)
weights_risk_parity = compute_risk_parity_weights(cov)
weights_monte_carlo = compute_monte_carlo_weights(cov)

# Make a results dataframe containing results from all the methods
results = weights_mean_varience.to_frame()
results = results.join(weights_risk_parity.to_frame())
results = results.join(weights_monte_carlo.to_frame())
print(results)

      Mean varience  Risk parity  Monte Carlo
AAPL       0.044085     0.094598     0.090909
ORCL       0.124387     0.108556     0.090909
EL         0.062053     0.076418     0.090909
NOW       -0.012220     0.054063     0.090909
ADI       -0.004111     0.081734     0.090909
ISRG      -0.007181     0.077688     0.090909
CVS        0.173233     0.124861     0.090909
T          0.299197     0.143724     0.090909
EOG        0.011555     0.042513     0.090909
REGN       0.222419     0.097072     0.090909
TJX        0.086583     0.098773     0.090909


# **PARAMETERS TO COMPARE THE STRATEGIES**

## **Expected Returns**

In [588]:
# Make a function to compute the expected returns which takes the weight matrix as an argument

def compute_ER(weights):
    mean = returns.mean(0)
    return weights.values * mean

In [589]:
# Pass the weight matrix generated by different methods to the 'compute_ER' function
# to get the expected returns

er_mv = compute_ER(weights_mean_varience)
er_mv.name = "Mean variance"
er_rp = compute_ER(weights_risk_parity)
er_rp.name = "Risk parity"
er_unif = compute_ER(weights_monte_carlo)
er_unif.name = "Monte Carlo"

In [590]:
# Make a seperate dataframe to store the expected returns generated by all the methods used

ers = er_mv.to_frame()
ers = pd.concat([ers,er_rp.to_frame()])
ers = pd.concat([ers,er_unif.to_frame()])
ers = ers.sum()
ers.name = "Expected Return"
ers = ers.to_frame()
print(ers)

               Expected Return
Mean variance         0.000555
Risk parity           0.000732
Monte Carlo           0.000789


## **Portfolio Volitality**

In [591]:
# Make a function to calculate the volitality of the portfolio generated
# by different methods which takes the weight matrix and covariance matrix as arguments

def portfolio_volatility(weights, cov):
    return np.sqrt(np.dot(np.dot(weights.values, cov.values), weights.values))

In [592]:
# Find the volitality of the various portfolio constructed by different methods

data = [portfolio_volatility(weights_mean_varience, cov)]
data.append(portfolio_volatility(weights_risk_parity, cov))
data.append(portfolio_volatility(weights_monte_carlo, cov))
volatility = pd.DataFrame(data = data, index=["Mean variance", "Risk parity", "Monte Carlo"],
columns=["Volatility"])

Assume the risk free return rate to be zero

In [593]:
def risk_free():
    return 0

## **Sharpe Ratio**

In [594]:
# Make a function to calculate the sharpe ratio of all the portfolios
# which takes the weight matrix and covariance matrix as arguments

def sharpe_ratio(weights, cov):
    ret_portfolio = compute_ER(weights).sum()
    ret_free = risk_free()
    volatility = portfolio_volatility(weights, cov)
    return (ret_portfolio - ret_free)/volatility

In [595]:
# Pass the weight and covariance matrix to the 'sharpe_ratio' function

data = [sharpe_ratio(weights_mean_varience, cov)]
data.append(sharpe_ratio(weights_risk_parity, cov))
data.append(sharpe_ratio(weights_monte_carlo, cov))
sharpe_R= pd.DataFrame(data = data, index=["Mean variance", "Risk parity", "Monte Carlo"],
columns=["Sharpe Ratio"])

# Print the Sharpe ratio dataframe
print(sharpe_R)

               Sharpe Ratio
Mean variance      0.043960
Risk parity        0.053527
Monte Carlo        0.054835


## **Maximum Drawdown**

In [596]:

def max_drawdown(returns):
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()
    return max_drawdown

data_drawdown = [max_drawdown(weights_mean_varience), max_drawdown(weights_risk_parity), max_drawdown(weights_monte_carlo)]
dd = pd.DataFrame(data=data_drawdown, index=["Mean variance", "Risk parity", "Monte Carlo"], columns=["Max DD"])


## **Profitability Ratio**

In [597]:
# Make a function to compute the Profitability Ratio for all the portfolios

def profitability_ratio(weights, cov):
    p_volatility = portfolio_volatility(weights, cov)
    return np.dot(np.sqrt(np.diag(cov.values)), weights) / p_volatility

In [598]:
# Pass the weights and covariance matrix to the 'diversification_ratio' function
# and store the results in a seperate dataframe 'dr'

data = [profitability_ratio(weights_mean_varience, cov)]
data.append(profitability_ratio(weights_risk_parity, cov))
data.append(profitability_ratio(weights_monte_carlo, cov))
pr = pd.DataFrame(data = data, index=["Mean variance", "Risk parity", "Monte Carlo"],
columns = ["Prof Ratio"])

## **FINAL RESULTS**

In [599]:
# Join all the dataframes constructed in one dataframe to get final results

final_results = ers.join(volatility)
final_results = final_results.join(sharpe_R)
final_results = final_results.join(dd)
final_results = final_results.join(pr)
print(final_results)

               Expected Return  Volatility  Sharpe Ratio    Max DD  Prof Ratio
Mean variance         0.000555    0.012630      0.043960 -0.023344    1.485430
Risk parity           0.000732    0.013682      0.053527  0.000000    1.508338
Monte Carlo           0.000789    0.014382      0.054835  0.000000    1.513450
